In [1]:
#if psycopg2 is not installed already then below
#!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [2]:
#this function will connect to AWS database
def connect_to_db(host_name,dbname,port,username,password):  
    try:
        conn = ps.connect(host=host_name,database=dbname,user=username,password=password,port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print('connected')
    return conn

In [3]:
# create table into database
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS CurrentWeather (
                            City VARCHAR(50) primary key,
                            Temperature VARCHAR(25) NOT NULL,
                            Pressure VARCHAR(25) NOT NULL,
                            Humidity VARCHAR(25) NOT NULL,
                            Wind  VARCHAR(25) NOT NULL;
                        ) """)
    curr.execute(create_table_command)

In [4]:
#check if city exist in table
def check_if_city_exist(curr, City):
    query = ("""SELECT City FROM CurrentWeather WHERE City=%s""")
    curr.execute(query,(City,))
    
    return curr.fetchone() is not None

In [5]:
#update row if city exist in table
def update_row(curr,City,Temperature,Pressure,Humidity,Wind):
    query = ("""UPDATE OneCall SET Temperature=%s,Pressure=%s,Humidity=%s,Wind=%s WHERE City =%s """)
    vars_to_update = (Temperature,Pressure,Humidity,Wind,City)
    curr.execute(query,vars_to_update)

In [7]:
#updating the database 
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns = ["City","Temperature","Pressure","Humidity","Wind speed"])

    for i, row in df.iterrows():
        if check_if_city_exist():
            update_row(curr,row['City'],row['Temperature'],row['Pressure'],row['Humidity'],row['Wind'])
        else:
            tmp_df = tmp_df.append(row)
    
    return tmp_df

In [8]:
#insert commond
def insert_into_table(curr,City,Temperature,Pressure,Humidity,Wind):
    insert_into_CurrentWeather = ("""INSERT INTO CurrentWeather (City,Temperature,Pressure,Humidity,WindCity,Temperature,Pressure,Humidity,Wind) 
                                    VALUES(%s,%s,%s,%s,%s.);""")
    row_to_insert = (City,Temperature,Pressure,Humidity,Wind)
    curr.execute(insert_into_CurrentWeather,row_to_insert)

In [9]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['City'],row['Temperature'],row['Pressure'],row['Humidity'],row['Wind'])

In [10]:
df = pd.read_csv('Current_Weather_API.csv',index_col=0)
df

,Temperature,Visibility,Wind speed,Clouds
0,84.0,10000.0,4.12,40.0
1,85.0,2500.0,4.12,75.0
2,74.0,10000.0,0.89,1.0
3,106.0,8000.0,4.63,0.0
4,66.0,10000.0,2.57,40.0
5,62.0,8047.0,1.79,90.0


In [ ]:
host_name = ""     #enter your host name
dbname = ""     #enter your database name
port = ''    #enter your port 
username = ''   #enter your username
password = ''   #enter your password
conn = None 

#this call will go to connect_to_db function and try to connect to database
conn = connect_to_db(host_name,dbname,port,username,password)

In [ ]:
curr = conn.cursor()

In [ ]:
#this will create the table into database
create_table(curr)

In [ ]:
new_city_df = update_db(curr, df)

In [ ]:
append_from_df_to_db(curr,new_city_df)

In [ ]:
conn.commit()